# Clase Landsat

In [ ]:
import os
import re
import rasterio
from urllib.request import urlopen


class NLandsat(object):
    
     
    '''Esta clase esta hecha para trabajar con la nueva Collection 2 de las imágenes Landsat, en concreto para
    utilizar las imágenes de la Collection 2 Level 2 (Surface Reflectance)'''
    
    
    
    def __init__(self, ruta):
        
        
        '''Instanciamos la clase con la escena que vayamos a procesar, hay que introducir todos los paths necesarios 
        como atributos del sistema. También al instanciar creamos un atributo referido a cada banda en cuanto a su 
        longitud de onda'''
        
        self.ruta_escena = ruta
        self.sr = os.path.split(self.ruta_escena)[0]
        self.escena = os.path.split(self.ruta_escena)[1]
        self.raiz = os.path.split(self.sr)[0]
        self.pro = os.path.join(self.raiz, 'Pro')
        
        if not os.path.exists(self.pro):
            os.makedirs(self.pro)
            
        self.pro_escena = os.path.join(self.pro, self.escena)
        
        if not os.path.exists(self.pro_escena):
            os.makedirs(self.pro_escena)
        
        escena_data = self.escena.split('_')
        
        #Landsat Name = ['LC08', 'L2SP', '202034', '20230125', '20230208', '02', 'T1']
        self.sat = escena_data[0]
        self.procesado = escena_data[1]
        self.path = escena_data[2][:3]
        self.row = escena_data[2][-3:]
        self.escena_date = escena_data[3]
        self.escena_process = escena_data[4]
        self.collection = escena_data[5]
        self.tier = escena_data[-1]
        
        self.mtl = {}
        for i in os.listdir(self.ruta_escena):
            #print i
            if i.endswith('MTL.txt'):
                mtl = os.path.join(self.ruta_escena,i)
                
                f = open(mtl, 'r') 

                #Dict
                for line in f.readlines(): 
                    if "=" in line: 
                        l = line.split("=") 
                        self.mtl[l[0].strip()] = l[1].strip()
                        
                        
        #Vamos a crear las variables con las bandas de la escena                
        if self.sat in ["LC08", "LC09"]:
            
            print('OLI sensor')
            for i in os.listdir(self.ruta_escena):
                #print(i)
                if re.search('TIF$', i):
                    #print(i)
                    
                    banda = os.path.splitext(i)[0].split('_')[-1]
                    #print(banda)

                    if banda == 'B2':
                        self.blue = os.path.join(self.ruta_escena, i)
                    elif banda == 'B3':
                        self.green = os.path.join(self.ruta_escena, i)
                    elif banda == 'B4':
                        self.red = os.path.join(self.ruta_escena, i)
                    elif banda == 'B5':
                        self.nir = os.path.join(self.ruta_escena, i)
                    elif banda == 'B6':
                        self.swir1 = os.path.join(self.ruta_escena, i)
                    elif banda == 'B7':
                        self.swir2 = os.path.join(self.ruta_escena, i)
                    elif banda == 'QA':
                        self.qa = os.path.join(self.ruta_escena, i)
                    else:
                        continue
                    
        elif self.sat in ['LE07', 'LT05']:

            for i in os.listdir(self.nor):
                if re.search('TIF$', i):
                    
                    banda = os.path.splitext(i)[0].split('_')[-1]
                                        
                    if banda == 'B1':
                        self.blue = os.path.join(self.ruta_escena, i)
                    elif banda == 'B2':
                        self.green = os.path.join(self.ruta_escena, i)
                    elif banda == 'B3':
                        self.red = os.path.join(self.ruta_escena, i)
                    elif banda == 'B4':
                        self.nir = os.path.join(self.ruta_escena, i)
                    elif banda == 'B5':
                        self.swir1 = os.path.join(self.ruta_escena, i)
                    elif banda == 'B7':
                        self.swir2 = os.path.join(self.ruta_escena, i)
                    elif banda == 'QA':
                        self.qa = os.path.join(self.ruta_escena, i)
                    else:
                        continue
                        
            else:
                
                print('No encuentro ninguna escena Landsat')
                        
        
        # Descargamos el Quicklook
        qlook_url = 'https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id={}'.format(self.mtl['LANDSAT_PRODUCT_ID'].strip('""'))
        print(qlook_url)
        qlook_name = 'Quicklook_' + self.escena
        qlook = open(os.path.join(self.ruta_escena, qlook_name), 'wb')
        u2=urlopen(qlook_url)
        junk=u2.read()
        qlook.write(junk)
        qlook.close()
        
        
        # Este sería el código para conectar con nuestra base de datos MongoDB (local) 
        self.newesc = {'_id': self.escena, 'usgs_id': self.mtl['LANDSAT_SCENE_ID'], 'tier_id': self.mtl['LANDSAT_PRODUCT_ID'],
                       'lpgs': self.mtl['PROCESSING_SOFTWARE_VERSION'], 'Clouds': {'cloud_scene': self.mtl['CLOUD_COVER']},
                       'Info': {'Tecnico': 'LAST-EBD Auto', 'Iniciada': datetime.now(), 'Pasos': {'rad': '', 'nor': ''}}}
        
        # Conectamos con MongoDB 
        connection = pymongo.MongoClient("mongodb://localhost")
        # DataBase: teledeteccion, Collection: landsat
        db=connection.teledeteccion
        landsat = db.landsat

        try:

            landsat.insert_one(self.newesc)

        except Exception as e:

            landsat.update_one({'_id':self.escena}, {'$set':{'Info.Iniciada': datetime.now()}})
            #print "Unexpected error:", type(e), se Podria dar un error por clave unica, por eso en
            #ese caso, lo que hacemos es actualizar la fecha en la que tratamos la imagen
            
        print('Landsat instanciada y subida a la base de datos')      
        
    

In [ ]:
path = '/media/diego/Datos3/EBD/Cursos/AETPython/Landsat/SR/LC08_L2SP_202034_20230125_20230208_02_T1'

a = NLandsat(path)